# Hands-on exercise with the JETSCAPE STAT package

Based on materials from Yi Chen - thanks!

We want to gain practical experience with the JETSCAPE STAT Bayesian analysis package.
In order to focus on the package rather than the physics, we'll investigate a simple toy model for the dijet asymmetry, $A_{\text{J}}$.
$A_{\text{J}}$ is defined as the difference between the two jets divided by the sum <a name="footnote-1"></a>[<sup>[1]</sup>](#footnote-1).
Specifically,

$$A_{\mathrm{J}} = \frac{p_{\mathrm{T, 1}} - p_{\mathrm{T, 2}}}{p_{\mathrm{T, 1}} + p_{\mathrm{T, 2}}}$$

Such an asymmetry is already apparent at the level of the event display

<center><img src="img/atlas_dijet_event_display.png" width=70% /></center>

Measured as $A_{\text{J}}$ in [Phys. Rev. Lett. 105 (2010) 252303](https://inspirehep.net/record/878733), with the PbPb data in black circles, pp in white circles, and HIJING in yellow

<center><img src="img/atlas_dijet_asymmetry_data.png" width=70% /></center>

<a name="footnote-1"></a>1. [^](#cite_ref-1) : (For experimental reasons related to removing background and detector effects with unfolding, $A_{\text{J}}$ has become somewhat less popular compared to $x_{\text{J}} = p_{\text{T, 2}}/p_{\text{T, 1}}$. However, it's convenient for thinking about conceptually, so we'll use it here.)

### Energy loss toy model

For today's exercise, We will construct a model to describe the energy loss observed in the dijet asymmetry.  For this model, we consider back-to-back dijets.  Each jet can lose energy, and the lost energy is parameterized as

$$ \Delta p_{\mathrm{T}} / p_{\mathrm{T}} \sim A \times Gaus(1, 1)$$

In addition to the energy loss contribution, we have extra "apparent" smearing directly applied to the $A_J$. This contribution comes from the fact that we have other processes going on in the events (three jets etc).  It is parameterized as a Gaussian smearing on $A_J$ with width $B$, centered at 0 (ie. for approximately balanced dijets). So there are two total parameters: $A$ and $B$.

For our toy, will we consider the measurement to be done in two bins of centrality.  One in central events, where ($A$, $B$) are both relevant, and another one in very peripheral events, where only the parameter ($B$) is relevant. To summarize:

|           | Central | Peripheral |
| --------- | ------- | ---------- |
| Quenching | Jets lose energy by $\Delta p_{\text{T}} / p_{\text{T}} \sim A \times \text{Gaus}(1, 1)$ | N/A |
| Smearing  | Smeared by $\text{Gaus}(0, B)$ | Smeared by $\text{Gaus}(0, B)$ |


## Step 0: Basic setup, loading python packages

Note that much (but not all!) of the code is parametrized, so it can be adapted to other datasets fairly easily

In [ ]:
import os
import pickle
from pathlib import Path

import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt

import src.reader as Reader

## Step 1: Input files

Performing a full scale Bayesian analysis often requires a good deal of organization.
Even if experimental measurements are on HEPdata, they are usually not formatted in a standard manner.
Files with design points and predictions may be in many different formats, making collaboration more difficult.

One of the benefit of the STAT package is the standardized input file formats. The full file specification is describe [here](https://www.evernote.com/l/ACWFCWrEcPxHPJ3_P0zUT74nuasCoL_DBmY). We will explore the different formats below

In [ ]:
# Helper functions for exploring input files

from typing import List, Sequence

def read_file_with_slices(filename: Path) -> List[str]:
    """Read file into list, allowing us to focus on particular slices.
    """
    with open(filename) as f:
        return [l for l in f]

def print_sequence_helper(seq: Sequence[str]) -> None:
    print("".join(seq))

In [ ]:
# Define some base paths for convenience
base_path = Path("input/AJExample")
img_base_path = Path("plots")
img_base_path.mkdir(exist_ok=True)

### Data files

Data files contain the measurements from experiments in a standardized yet flexible format.
Usually, you will acquire these files from HEPdata or directly from an experiment.
In the case of our toy model, we provide the files directly (since we've generating them according to a toy)

Each file contains general header info, specifies the columns, and the data itself. For the example, I show only the central values

In [ ]:
example_data_file = read_file_with_slices(base_path / "Data_Selection1.dat")
# First, take a glance at the file
print_sequence_helper(example_data_file[:15])

General header info, covering where it came from (experiment, collision system, centrality, etc)

In [ ]:
print_sequence_helper(example_data_file[0:7])

Header specification of the columns. Here, we have the x bin edges, the central value, the lower and upper statistical error, and the lower and upper systematic error. This can also include additional classes of uncertainties

In [ ]:
print_sequence_helper(example_data_file[7:8])

Finally, the data itself. We'll just take a look at a quick subset for brevity (repeating the column header)

In [ ]:
print_sequence_helper(example_data_file[7:15])

### Design points

Documents the parameter values that are used for running the model. Can distribute them in many different ways, including taking advantage of machine learning techniques such as active learning to attempt to optimize calculations

In [ ]:
example_design_points_file = read_file_with_slices(base_path / "Design.dat")
# First, take a glance at the file
print_sequence_helper(example_design_points_file[:5])

Header info, specifying the relevant parameters. Note that you need to keep track of your generation range (this could be included in the header. Here we specify the minimum, but adding more doesn't hurt). However many you generate depends on many factors, including the parameterizaton. At least 10 / parameter is a reasonable start

In [ ]:
print_sequence_helper(example_design_points_file[0:2])

The design points themselves. One row per set of parameters

In [ ]:
print_sequence_helper(example_design_points_file[2:10])

### Prediction files

These files contain the predictions run at the design points specified in the previous file. 

In [ ]:
example_predictions_file = read_file_with_slices(base_path / "Prediction_Selection1.dat")
# First, take a glance at the file
print_sequence_helper(example_predictions_file[:4])

Header information includes the specification, the corresponding data file, and the design points used to generate the predictions

In [ ]:
print_sequence_helper(example_predictions_file[0:3])

The predictions themselves.  Each row is a prediction for **one data point from all design points**. Here, we generated 40 design points, so there are 40 values in each row. The rows match up with the binning in the data file

In [ ]:
print_sequence_helper(example_predictions_file[3:4])
print(f"Number of design points: {len(example_predictions_file[3:4][0].split(' '))}")

### Load data, design points, and predictions from text files

The `Reader` class implements the interface between input data files and the code.  There are functions to read in experimental data, externally-supplied covariance matrix, design points, and calculations for the design points

In [ ]:
# Read data files
RawData1 = Reader.ReadData(base_path / "Data_Selection1.dat")   # Central bin data
RawData2 = Reader.ReadData(base_path / "Data_Selection2.dat")   # Peripheral bin data

# Read covariance -- if you have covariance matrix from the experiments for example
# We will estimate the covariance separately below
# RawCov1 = Reader.ReadCovariance('input/AJExample/Covariance_Selection1_TypeX.dat')
# RawCov2 = Reader.ReadCovariance('input/AJExample/Covariance_Selection2_TypeX.dat')

# Read design points
RawDesign = Reader.ReadDesign(base_path / "Design.dat")   # Design points!

# Read model prediction
# Here, these are our toy calculations
RawPrediction1 = Reader.ReadPrediction(base_path / "Prediction_Selection1.dat")   # Calculation for central bin
RawPrediction2 = Reader.ReadPrediction(base_path / "Prediction_Selection2.dat")   # Calculation for peripheral

This stores everything into an expected dictionary format, which is easily convertable and consumable by the STAT package

In addition to reading inputs, there is also a `EstimateCovariance` function that estimates the covariance for you.  It is configurable to do different correlation treatment for different systematic uncertainty sources (and then add them all up!).  We can also correlate across different measurements.

### Setup analysis

We've read the inputs, but we still need to compile all of the inputs into a format that can be understood by the STAT package.
This problem is fairly flexible, so it's not always straightforwad to automate.
By configuring by hand, we can be confident that we're working with the correct inputs

First, we begin with the basic setup

In [ ]:
# Initialize empty dictionary
# The package expects all inputs to be in a properly formatted dictionary.
# This dict is maintained as ~a global object
AllData = {}

# We have two different centrality bins, so we need to tell the package about them.
C0 = 'Central'
C1 = 'Peripheral'

# Basic information
AllData["systems"] = ["PbPb5020"]                 # List of collision systems we are doing
AllData["keys"] = RawDesign["Parameter"]          # Get the "A" and "B" from the design file
AllData["labels"] = RawDesign["Parameter"]        # Get the "A" and "B" from the design file
AllData["ranges"] = [(0, 0.3), (0, 0.3)]          # Range of A and B. You need this information externally,
                                                  # or to store it in the design points header.
AllData["observables"] = [('A_J', [C0, C1])]      # We measure A_J with two bins: "Central" and "Peripheral"
# In this example "C0" is central data, and "C1" is peripheral => see above


With the basic structure configured, we can now start packaging up our input into the matching format

In [ ]:

# Data points
Data = {"PbPb5020": {"A_J": {C0: RawData1["Data"], C1: RawData2["Data"]}}}

# Model predictions
# We provide the binning from the experimental data. It's critical that this matches!
Prediction = {"PbPb5020": {"A_J": {C0: {"Y": RawPrediction1["Prediction"], "x": RawData1["Data"]['x']},
                                   C1: {"Y": RawPrediction2["Prediction"], "x": RawData2["Data"]['x']}}}}


Next, we can estimate the correlations among uncertainties, as decribed by Yi

In [ ]:
# Correlation length options
# Change to 9999 to make it fully correlated,or -1 for fully uncorrelated
# Also try 0.1 for partially correlated
CorrelationLength = -1
OffDiagonalCorrelationLength = -1

# Covariance matrices - the indices are [system][measurement1][measurement2], each one is a block of matrix
Covariance = Reader.InitializeCovariance(Data)
Covariance["PbPb5020"][("A_J", C0)][("A_J", C0)] = \
    Reader.EstimateCovariance(RawData1, RawData1, SysLength = {"default": CorrelationLength}, ScaleX = False)
Covariance["PbPb5020"][("A_J", C1)][("A_J", C1)] = \
    Reader.EstimateCovariance(RawData2, RawData2, SysLength = {"default": CorrelationLength}, ScaleX = False)
Covariance["PbPb5020"][("A_J", C0)][("A_J", C1)] = \
    Reader.EstimateCovariance(RawData1, RawData2, SysLength = {"default": OffDiagonalCorrelationLength}, ScaleX = False)
Covariance["PbPb5020"][("A_J", C1)][("A_J", C0)] = \
    Reader.EstimateCovariance(RawData2, RawData1, SysLength = {"default": OffDiagonalCorrelationLength}, ScaleX = False)

After all of that preparation, store everying in the global dict. The package expects this to be stored in `input/default.p`, so take care if working on multiple projects at once!

In [ ]:
# Assign data to the dictionary
AllData["design"] = RawDesign["Design"]
AllData["model"] = Prediction
AllData["data"] = Data
AllData["cov"] = Covariance

# Save to the desired file
with open('input/default.p', 'wb') as handle:
    pickle.dump(AllData, handle, protocol = pickle.HIGHEST_PROTOCOL)

## Explore our input files

Let's take a look at what we're working with!

In [ ]:
# First, let's take a look at the input data!

# Setup
system_count = len(AllData["systems"])
figure, axes = plt.subplots(figsize = (15, 5 * system_count), ncols = 2, nrows = 1)

# We only have one observable, the AJ, so we always take the first entry
# If there were more, we would have to iterate over this index too (and likely adjust the sub_observable_index)
observable_index = 0

for sub_observable_index, ax in enumerate(axes):
    ax.set_xlabel(r"$A_{J}$")
    ax.set_ylabel(r"$dN/dA_{J}$")

    # Extract some information
    system_name = AllData["systems"][0]
    # Name (key) of the first observable.
    observable_name = AllData["observables"][observable_index][0]
    # The second entry in the tuple is a list of the observable names
    centrality = AllData["observables"][observable_index][1][sub_observable_index]

    # Actual bins and data points
    DX = AllData["data"][system_name][observable_name][centrality]['x']
    DY = AllData["data"][system_name][observable_name][centrality]['y']
    # Combine the statistical and systematic errors in quadrature
    DE = np.sqrt(
        AllData["data"][system_name][observable_name][centrality]['yerr']['stat'][:,0]**2
        + AllData["data"][system_name][observable_name][centrality]['yerr']['sys'][:,0]**2
    )
                
    ax.errorbar(DX, DY, yerr = DE, fmt='ro', label="Measurements")

axes[0].set_title('Central')
axes[1].set_title('Peripheral')

plt.tight_layout()
figure.savefig(img_base_path / 'InputData.pdf', dpi = 192)
# figure

Next, we can look at the design points. We want to aim to cover our parameter space

In [ ]:
# Plot 2D scatter plot of the design points
figure, axis = plt.subplots(figsize = (5, 5), ncols = 1, nrows = 1)
axis.plot(AllData["design"][:,0], AllData["design"][:,1], 'o')
axis.set_xlabel('A')
axis.set_ylabel('B')
figure.savefig(img_base_path / 'DesignPoints.pdf', dpi = 192)

Next, we can compare the calculations to the input data

In [ ]:
# Grab the model predictions for convenience
_model_predictions = AllData["model"]

figure, axes = plt.subplots(figsize = (15, 5 * system_count), ncols = 2, nrows = 1)

# We only have one observable, the AJ, so we always take the first entry
# If there were more, we would have to iterate over this index too (and likely adjust the sub_observable_index)
observable_index = 0

for sub_observable_index, ax in enumerate(axes):
    ax.set_xlabel(r"$A_{J}$")
    ax.set_ylabel(r"$dN/dA_{J}$")

    # Extract some information
    system_name = AllData["systems"][0]
    # Name (key) of the first observable.
    observable_name = AllData["observables"][observable_index][0]
    # The second entry in the tuple is a list of the observable names
    centrality = AllData["observables"][observable_index][1][sub_observable_index]

    # Actual bins and data points
    DX = AllData["data"][system_name][observable_name][centrality]['x']
    DY = AllData["data"][system_name][observable_name][centrality]['y']
    # Combine the statistical and systematic errors in quadrature
    DE = np.sqrt(
        AllData["data"][system_name][observable_name][centrality]['yerr']['stat'][:,0]**2
        + AllData["data"][system_name][observable_name][centrality]['yerr']['sys'][:,0]**2
    )
                
    for i, y in enumerate(_model_predictions[system_name][observable_name][centrality]['Y']):
        ax.plot(DX, y, 'b-', alpha=0.1, label="Posterior" if i==0 else '')
    ax.errorbar(DX, DY, yerr = DE, fmt='ro', label="Measurements")

axes[0].set_title('Central')
axes[1].set_title('Peripheral')
    
plt.tight_layout()
figure.savefig(img_base_path / 'Design.pdf', dpi = 192)
# figure

Visualize the covariance matrices

In [ ]:
# Need the maximum value from the different covariances so that we can put all plots on the same scale
MaxValue = np.array(
    [Covariance["PbPb5020"][("A_J", C0)][("A_J", C0)].max(),
     Covariance["PbPb5020"][("A_J", C0)][("A_J", C1)].max(),
     Covariance["PbPb5020"][("A_J", C1)][("A_J", C0)].max(),
     Covariance["PbPb5020"][("A_J", C1)][("A_J", C1)].max()]).max()

figure, axes = plt.subplots(figsize = (6, 6), ncols = 2, nrows = 2)
axes[0][0].imshow(Covariance["PbPb5020"][("A_J", C0)][("A_J", C0)], vmin=0, vmax=MaxValue)
axes[0][1].imshow(Covariance["PbPb5020"][("A_J", C0)][("A_J", C1)], vmin=0, vmax=MaxValue)
axes[1][0].imshow(Covariance["PbPb5020"][("A_J", C1)][("A_J", C0)], vmin=0, vmax=MaxValue)
axes[1][1].imshow(Covariance["PbPb5020"][("A_J", C1)][("A_J", C1)], vmin=0, vmax=MaxValue)

axes[0][0].set_xlabel('Central')
axes[0][1].set_xlabel('Central')
axes[1][0].set_xlabel('Peripheral')
axes[1][1].set_xlabel('Peripheral')

axes[0][0].set_ylabel('Central')
axes[0][1].set_ylabel('Peripheral')
axes[1][0].set_ylabel('Central')
axes[1][1].set_ylabel('Peripheral')

figure.tight_layout()
figure.savefig(img_base_path / 'Covariance.pdf', dpi = 192)

These reflect the choices we made above in terms of estimate the covariance

### Clean past files so that they don't haunt us later on

This is relevant due to the global state of the package. Don't worry too much about the details here. Just run it and keep in mind that this is a good thing to check if you run into an issue.

In [ ]:
# Clean past MCMC samples
mcmc_samples_cache = Path('cache/mcmc_chain.hdf')
if mcmc_samples_cache.exists():
    mcmc_samples_cache.unlink()

# Clean past emulator
for system in AllData["systems"]:
    emulator_cache = Path("cache/emulator") / f"{system}.pkl"
    if emulator_cache.exists():
        emulator_cache.unlink()

## Step 2: Train the GP emulator

We want to train the Gaussian Process emulator to cover the prediction the entire phase space.
It will take the inputs that we've stored in the global dict, and train the emulator, utilzing some set number of principal components.
Here, we've found 8 to be sufficient to describe 99.5% of the variance, but you could tune this further

In [ ]:
# See the emulator options
! python3 -m src.emulator --help

In [ ]:
# Train emulator with 8 principle components
# (it implicitly knowns to run on PbPb5020)
! python3 -m src.emulator --retrain --npc 8

We've now trained the emulator, and the results were only written to file (`cache/emulator/PbPb5020.pkl`).
As the filename implies, we often train a different emulator for each $\sqrt{s}$.
Since we want to work with the emulator directly for plotting purposes, we'll load it now

In [ ]:
# Read in emulator object into this notebook for plotting purposes
from src import emulator
EmulatorPbPb5020 = emulator.Emulator.from_cache('PbPb5020')

Now we can experiment with the emulator.
Let's try out a few predictions to see if they make sense.
eg. [0.0, 0.25], [0.1, 0.0], [0.2, 0.1], etc

Note the maximum range is 0.3.  What will happen if you put something larger than 0.3?  For example [0.1, 0.5]

In [ ]:
# Setup
_prediction = {"PbPb5020": EmulatorPbPb5020.predict([[0.0, 0.25]])}
system_count = len(AllData["systems"])

figure, axes = plt.subplots(figsize = (15, 5 * system_count), ncols = 2, nrows = 1)

# We only have one observable, the AJ, so we always take the first entry
# If there were more, we would have to iterate over this index too (and likely adjust the sub_observable_index)
observable_index = 0

for sub_observable_index, ax in enumerate(axes):
    ax.set_xlabel(r"$A_{J}$")
    ax.set_ylabel(r"$dN/dA_{J}$")

    # Extract some information
    system_name = AllData["systems"][0]
    # Name (key) of the first observable.
    observable_name = AllData["observables"][observable_index][0]
    # The second entry in the tuple is a list of the observable names
    centrality = AllData["observables"][observable_index][1][sub_observable_index]

    # Actual bins and data points
    DX = AllData["data"][system_name][observable_name][centrality]['x']
    DY = AllData["data"][system_name][observable_name][centrality]['y']
    # Combine the statistical and systematic errors in quadrature
    DE = np.sqrt(
        AllData["data"][system_name][observable_name][centrality]['yerr']['stat'][:,0]**2
        + AllData["data"][system_name][observable_name][centrality]['yerr']['sys'][:,0]**2
    )
                
    ax.plot(DX, _prediction[system_name][observable_name][centrality][0], 'b-', alpha=1, label="Posterior")
    ax.errorbar(DX, DY, yerr = DE, fmt='ro', label="Measurements")

axes[0].set_title('Central')
axes[1].set_title('Peripheral')

axes[1].legend(frameon=False, fontsize=16, loc="upper right")
figure.tight_layout()
figure.savefig(img_base_path / 'PredictionTest.pdf', dpi = 192)
# figure

## Step 3: MCMC sampling

Using the emulator, we can sample the posterior with MCMC

In [ ]:
! python3 -m src.mcmc --help

Here, we will run 100 walkers (chains), with 200 burn in steps. We'll then run 200 steps further.
For a real analysis, you'll need much more than this!

In [ ]:
# First, make sure that our cache is clean before we try to sample
if mcmc_samples_cache.exists():
    mcmc_samples_cache.unlink()
! python3 -m src.mcmc --nwalkers 100 --nburnsteps 200 200

Much like the emulator, the MCMC writes the chain to file.
To work with it, we need to explicitly load the posterior samples into the notebook

In [ ]:
# Now important the main package. This is important, because it gives us direct access to the global state
import src
src.Initialize()
from src import mcmc
chain = mcmc.Chain()
MCMCSamples = chain.load()

## Step 4: Explore the MCMC and the posterior

Assuming all of the above worked, we're now at the point where we can evaluate the MCMC performance, our most probable paramteers, as well as how well our posterior describes our data.

To start, let's take a quick look at the MCMC performance

In [ ]:
# Walker vs step
names = AllData["labels"]

with chain.dataset() as d:
    W = d.shape[0]     # number of walkers
    S = d.shape[1]     # number of steps
    N = d.shape[2]     # number of parameters
    T = int(S / 100)   # "thinning" -> plot only a subset of the walkers to keep the plot interpretable
    # Adjsut the alpha based on how many walks we will look at
    img_alpha = 20 / W
    figure, axes = plt.subplots(figsize = (15, 2 * N), ncols = 1, nrows = N)
    for i, ax in enumerate(axes):
        ax.set_ylabel(names[i])
        ax.set_xlabel('Step')
        for j in range(0, W):
            ax.plot(range(0, S, T), d[j, ::T, i], alpha = img_alpha)
    figure.tight_layout()
    plt.savefig(img_base_path / 'MCMCSamples.pdf', dpi = 192)

Now, we can take a look at the extracted parameters!

In [ ]:
NDimension = len(AllData["labels"])
Ranges = np.array(AllData["ranges"]).T
figure, axes = plt.subplots(figsize = (3 * NDimension, 3 * NDimension), ncols = NDimension, nrows = NDimension)
Names = AllData["labels"]
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        if i==j:
            ax.hist(MCMCSamples[:,i], bins=50,
                    range=Ranges[:,i], histtype='step', color='green')
            ax.set_xlabel(Names[i])
            ax.set_xlim(*Ranges[:,j])
        if i>j:
            ax.hist2d(MCMCSamples[:, j], MCMCSamples[:, i], 
                      bins=50, range=[Ranges[:,j], Ranges[:,i]], 
                      cmap='Greens')
            ax.set_xlabel(Names[j])
            ax.set_ylabel(Names[i])
            ax.set_xlim(*Ranges[:,j])
            ax.set_ylim(*Ranges[:,i])
        if i<j:
            ax.axis('off')
figure.tight_layout()
plt.savefig(img_base_path / 'Correlation.pdf', dpi = 192)
# figure

Posterior is quite well constrained for our simple toy model!

However, this isn't enough. How does the posterior describe the data?  

In [ ]:
# Grab some of the MCMC samples. This is sampling the posterior
_examples = MCMCSamples[ np.random.choice(range(len(MCMCSamples)), 2000), :]
_prediction = {"PbPb5020": EmulatorPbPb5020.predict(_examples)}
_system_count = len(AllData["systems"])

figure, axes = plt.subplots(figsize = (15, 5 * _system_count), ncols = 2, nrows = 1)

# We only have one observable, the AJ, so we always take the first entry
# If there were more, we would have to iterate over this index too (and likely adjust the sub_observable_index)
observable_index = 0

for sub_observable_index, ax in enumerate(axes):
    ax.set_xlabel(r"$A_{J}$")
    ax.set_ylabel(r"$dN/dA_{J}$")

    # Extract some information
    system_name = AllData["systems"][0]
    # Name (key) of the first observable.
    observable_name = AllData["observables"][observable_index][0]
    # The second entry in the tuple is a list of the observable names
    centrality = AllData["observables"][observable_index][1][sub_observable_index]

    # Actual bins and data points
    DX = AllData["data"][system_name][observable_name][centrality]['x']
    DY = AllData["data"][system_name][observable_name][centrality]['y']
    # Combine the statistical and systematic errors in quadrature
    DE = np.sqrt(
        AllData["data"][system_name][observable_name][centrality]['yerr']['stat'][:,0]**2
        + AllData["data"][system_name][observable_name][centrality]['yerr']['sys'][:,0]**2
    )
                
    for i, y in enumerate(_prediction[system_name][observable_name][centrality]):
        ax.plot(DX, y, 'b-', alpha=0.005, label="Posterior" if i==0 else '')
    ax.errorbar(DX, DY, yerr = DE, fmt='ro', label="Measurements")

axes[0].set_title('Central')
axes[1].set_title('Peripheral')

axes[1].legend(frameon=False, fontsize=16, loc="upper right")
plt.tight_layout()
figure.savefig(img_base_path / 'ObservablePosterior.pdf', dpi = 192)
# figure

Our posterior describes our toy data quite well (as we would hope!)

In [ ]:
# close all plots to save memory
plt.close('all')

We've made it!

Let's take a look back at the correlations between the uncertainties. For example, what happens for if they're both highly correlated? This can be have a critical impact on your result